In [2]:
#load libraries
import os,glob
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

In [3]:
#load data: selecting KatyPerry, LMFAO, Eminem, Shakira

# merging csv files
df = pd.concat(
    map(pd.read_csv, ['Youtube02-KatyPerry.csv', 'Youtube03-LMFAO.csv', 'Youtube04-Eminem.csv', 'Youtube05-Shakira.csv']), ignore_index=True)
#preview data
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,z12pgdhovmrktzm3i23es5d5junftft3f,lekanaVEVO1,2014-07-22T15:27:50,i love this so much. AND also I Generate Free ...,1
1,z13yx345uxepetggz04ci5rjcxeohzlrtf4,Pyunghee,2014-07-27T01:57:16,http://www.billboard.com/articles/columns/pop-...,1
2,z12lsjvi3wa5x1vwh04cibeaqnzrevxajw00k,Erica Ross,2014-07-27T02:51:43,Hey guys! Please join me in my fight to help a...,1
3,z13jcjuovxbwfr0ge04cev2ipsjdfdurwck,Aviel Haimov,2014-08-01T12:27:48,http://psnboss.com/?ref=2tGgp3pV6L this is the...,1
4,z13qybua2yfydzxzj04cgfpqdt2syfx53ms0k,John Bello,2014-08-01T21:04:03,Hey everyone. Watch this trailer!!!!!!!! http...,1


In [4]:
#clean comment data

#convert comments to string type
df['CONTENT'] = df['CONTENT'].astype(str)
#lower case all comments
df['CONTENT'] = df['CONTENT'].str.lower()
#remove punctuation
df['CONTENT'] = df['CONTENT'].str.replace('[^\w\s]','')
#remove numbers
df['CONTENT'] = df['CONTENT'].str.replace('\d+', '')
#remove extra spaces
df['CONTENT'] = df['CONTENT'].str.replace('\s+', ' ')
#include only English alphabet
df[df['CONTENT'].map(lambda x: x.isascii())]
#remove stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
df['CONTENT'] = df['CONTENT'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

#how many of each class?
df['CLASS'].value_counts()

/var/folders/t6/96kvh0194vl8gsjz443_h2f80000gn/T/ipykernel_30562/3586069163.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['CONTENT'] = df['CONTENT'].str.replace('[^\w\s]','')
/var/folders/t6/96kvh0194vl8gsjz443_h2f80000gn/T/ipykernel_30562/3586069163.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['CONTENT'] = df['CONTENT'].str.replace('\d+', '')
/var/folders/t6/96kvh0194vl8gsjz443_h2f80000gn/T/ipykernel_30562/3586069163.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['CONTENT'] = df['CONTENT'].str.replace('\s+', ' ')
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/laurensantos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1    830
0    776
Name: CLASS, dtype: int64

In [5]:
#transform data
cv = CountVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words=None, ngram_range=(1,1), analyzer='word')
dt_mat = cv.fit_transform(df.CONTENT)

tfidf_transformer = TfidfTransformer()
tfidf_mat = tfidf_transformer.fit_transform(dt_mat)

words = pd.DataFrame(dt_mat.todense(), index=df.index, columns=cv.get_feature_names())
words['CLASS'] = df.CLASS
#print(words)

#there are 3416 unique words in this data set

/Users/laurensantos/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [7]:
X = words.iloc[:, 0:3416]
y = words.iloc[: , -1:]

#train/test data using 70/30 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

#using multinomial distribution; discrete counting of words

mnb = MultinomialNB()
mnb.fit(X_train, y_train.values.ravel())

y_pred = mnb.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.8941908713692946
[[201  18]
 [ 33 230]]


In [8]:
#test on remaining dataset 
psy_df = pd.read_csv('Youtube01-Psy.csv')

#preview
psy_df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [11]:
#clean comment data

#convert comments to string type
psy_df['CONTENT'] = psy_df['CONTENT'].astype(str)
#lower case all comments
psy_df['CONTENT'] = psy_df['CONTENT'].str.lower()
#remove punctuation
psy_df['CONTENT'] = psy_df['CONTENT'].str.replace('[^\w\s]','')
#remove numbers
psy_df['CONTENT'] = psy_df['CONTENT'].str.replace('\d+', '')
#remove extra spaces
psy_df['CONTENT'] = psy_df['CONTENT'].str.replace('\s+', ' ')
#include only English alphabet
psy_df[psy_df['CONTENT'].map(lambda x: x.isascii())]
#remove stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
psy_df['CONTENT'] = psy_df['CONTENT'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

#how many of each class?
psy_df['CLASS'].value_counts()
#there are 170 in each class; even split

/var/folders/t6/96kvh0194vl8gsjz443_h2f80000gn/T/ipykernel_30562/2101640157.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  psy_df['CONTENT'] = psy_df['CONTENT'].str.replace('[^\w\s]','')
/var/folders/t6/96kvh0194vl8gsjz443_h2f80000gn/T/ipykernel_30562/2101640157.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  psy_df['CONTENT'] = psy_df['CONTENT'].str.replace('\d+', '')
/var/folders/t6/96kvh0194vl8gsjz443_h2f80000gn/T/ipykernel_30562/2101640157.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  psy_df['CONTENT'] = psy_df['CONTENT'].str.replace('\s+', ' ')
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/laurensantos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1    175
0    175
Name: CLASS, dtype: int64

In [14]:
#transform data
cv2 = CountVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words=None, ngram_range=(1,1), analyzer='word')
dt_mat2 = cv2.fit_transform(psy_df.CONTENT)

tfidf_transformer = TfidfTransformer()
tfidf_mat2 = tfidf_transformer.fit_transform(dt_mat2)

words2 = pd.DataFrame(dt_mat2.todense(), index=psy_df.index, columns=cv2.get_feature_names())
words2['CLASS'] = psy_df.CLASS
#print(words2)
#there are 1097 unique words

/Users/laurensantos/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
X = words2.iloc[:, 0:1097]
y = words2.iloc[: , -1:]

#using multinomial distribution; discrete counting of words

mnb = MultinomialNB()
mnb.fit(X, y.values.ravel())

y_pred = mnb.predict(X)

print(accuracy_score(y, y_pred))
print(confusion_matrix(y, y_pred))

0.9914285714285714
[[173   2]
 [  1 174]]
